In [65]:
#Importing packages
import pandas as pd
import seaborn as sns
import time
import re
import numpy as np
from langdetect import detect
from googletrans import Translator, constants

In [5]:
dframe_general_info = pd.read_excel('29_companies_essential_data.xlsx',sheet_name='29_companies_essential_data')
dframe_jobs = pd.read_excel('29_companies_essential_data.xlsx',sheet_name='jobs')
dframe_jobs_final = dframe_jobs.iloc[:,:15]

In [6]:
dframe_jobs_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   company_name               112 non-null    object        
 1   job_area                   107 non-null    object        
 2   job_link                   109 non-null    object        
 3   job_extraction_timestamp   112 non-null    datetime64[ns]
 4   job_about_company          100 non-null    object        
 5   job_title                  106 non-null    object        
 6   job_location               106 non-null    object        
 7   job_description            84 non-null     object        
 8   job_main_responsabilities  105 non-null    object        
 9   job_other_details          101 non-null    object        
 10  is_PCD_mentioned           79 non-null     object        
 11  PCD_text                   52 non-null     object        
 12  is_diver

Firstly we will use 'langdetect' library to determine the language for columns job_description, job_main_responsabilities, and job_other_details.

In [7]:
#This is just a test on how 'detect' from library langdetect works
print("This job description below: \n {text}".format(text=str(dframe_jobs_final.iloc[0].to_dict()['job_description'])))
print("\n")
print("Is recognized by langdetect as being in: \n {lang}".format(lang=str(detect(dframe_jobs_final.iloc[0].to_dict()['job_description']))))

This job description below: 
 We are looking for a seasoned FP&A Manager that will have the critical mission of supporting our Food delivery exponential growth by driving the budget, forecast, actual, analyzes, insights. This is a unique opportunity to generate impact within an fast-growing business. You will be a trusted business partner and own the managing margins, finance metrics and EBT.


Is recognized by langdetect as being in: 
 en


In [8]:
#Creating a function that receives a list of texts and returns another list evalutating in what language texts were written
def detect_lang(list_of_terms):
    list_lang = []
    for x in list_of_terms:
        try:
            list_lang.append(detect(x))
        except:
            list_lang.append('No_text')
    return list_lang

In [9]:
#Creating columns with lang detection inside the original dframe
dframe_jobs_final['job_description_lang'] = detect_lang(dframe_jobs_final['job_description'].to_list())
dframe_jobs_final['job_main_responsabilities_lang'] = detect_lang(dframe_jobs_final['job_main_responsabilities'].to_list())
dframe_jobs_final['job_other_details_lang'] = detect_lang(dframe_jobs_final['job_other_details'].to_list())

In [10]:
#Now let's check if we have anomalies in terms of different languages between columns
dframe_jobs_final.groupby(['job_description_lang','job_main_responsabilities_lang','job_other_details_lang'], as_index=False)['job_extraction_timestamp'].count()

,job_description_lang,job_main_responsabilities_lang,job_other_details_lang,job_extraction_timestamp
0,No_text,No_text,No_text,6
1,No_text,en,en,8
2,No_text,pt,No_text,2
3,No_text,pt,pt,12
4,en,en,No_text,1
5,en,en,en,33
6,es,es,es,1
7,pt,No_text,pt,1
8,pt,pt,No_text,2
9,pt,pt,pt,46


We have 6 rows without info; all the other rows are filled with one column at least, and the language makes sense.

In [11]:
#In order to evaluate accountant profile we will use the rows where job_main_responsabilities AND job_other_details are filled simultaneously
idx = np.where((dframe_jobs_final['job_main_responsabilities'].notnull()) & (dframe_jobs_final['job_other_details'].notnull()))
dframe_jobs_final = dframe_jobs_final.iloc[idx]

In [12]:
dframe_jobs_final.groupby(['job_description_lang','job_main_responsabilities_lang'], as_index=False)['job_extraction_timestamp'].count()

,job_description_lang,job_main_responsabilities_lang,job_extraction_timestamp
0,No_text,en,8
1,No_text,pt,12
2,en,en,33
3,es,es,1
4,pt,pt,46


As we can see: <br> # there are plenty job_descriptions and main_responsabilities in english and 1 is in spanish, we need all of them to be in portuguese

In [29]:
def unescape(s):
    s = s.replace("&lt;", "<")
    s = s.replace("&gt;", ">")
    # this has to be last:
    s = s.replace("&amp;", "&")
    return s

In [13]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="apikey.json"

In [30]:
##It must be with pip install google-cloud-translate==2.0.1 !!!!!
def translate_text(target, text):
    """Translates text into the target language.

    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """
    import six
    from google.cloud import translate_v2 as translate

    translate_client = translate.Client()

    #if isinstance(text, six.binary_type):
     #   text = text.decode("Windows-1252")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target)

    
    return unescape(result["translatedText"])

In [31]:
#Testing something in english
dframe_jobs_final.iloc[0]['job_main_responsabilities']

'-Detailed analysis of P&L variances versus forecast, budget and actual. Understand and explain variances to the Regional leadership, strong stakeholder management.\n-Full P&L Forecast in a monthly basis to the HQ\n-Prepare monthly forecast adjustments to both P&L and Balance Sheet; import, analyze, prepare and export data and reports \n- Responsible for the accuracy, consolidation, conciliation and load of data into BI as needed for Brazil\n-Prepare, consolidate and analyze the Annual Budget\n-Analysis of main deviations and financial impacts, understanding of root causes and develop action plans\n- Prepare and own executive presentations to HQ '

In [32]:
#How it translates to pt: WORKS GREAT!
translate_text(target='pt-br',text=str(dframe_jobs_final.iloc[0]['job_main_responsabilities']))

'-Análise detalhada das variações de P&L versus previsão, orçamento e real. Compreenda e explique as variações para a liderança regional, forte gerenciamento de partes interessadas. -Previsão completa de P&L mensalmente para o HQ -Preparar ajustes de previsão mensal para P&L e Balanço Patrimonial; importar, analisar, preparar e exportar dados e relatórios - Responsável pela precisão, consolidação, conciliação e carregamento de dados em BI conforme necessário para o Brasil -Preparar, consolidar e analisar o Orçamento Anual -Análise dos principais desvios e impactos financeiros, entendimento da raiz causas e desenvolver planos de ação - Preparar e apresentar apresentações executivas para HQ'

In [40]:
#Now let's create another function that receives a list and translate it's items to portuguese when needed
def translate_list(text_list):
    translated_result=[]
    for item in text_list:
        if str(detect(item))=='pt':
            translated_result.append(item)
        else:
            translated_result.append(translate_text(target='pt-br',text=str(item)))
    return translated_result

In [46]:
#Translating job_main_responsabilities
job_main_responsabilities_pt_list = translate_list(dframe_jobs_final['job_main_responsabilities'].to_list())

In [47]:
#Translating job_other_details
job_other_details_pt_list = translate_list(dframe_jobs_final['job_other_details'].to_list())

In [48]:
#Adding translated columns into dframe
dframe_jobs_final['job_main_responsabilities_pt'] = job_main_responsabilities_pt_list
dframe_jobs_final['job_other_details_pt'] = job_other_details_pt_list

In [53]:
#This is how our dataframe looks so far
dframe_jobs_final.iloc[0].to_dict()

{'company_name': 99,
 'job_area': 'Finance',
 'job_link': 'https://carreiras.99app.com/jobdetail/?vagaid=d0e94c27-ee68-4205-b68c-44d1720244a8',
 'job_extraction_timestamp': Timestamp('2021-06-27 11:38:13.363000'),
 'job_about_company': "99 is a Brazilian technology company founded in 2012 and today is part of the global company DiDi Chuxing, the largest app platform for convenience and transport in the world. Here, our mission is to build the best journey for users, transforming people's lives by offering more accessible, safe and convenient services for passengers and customers, and the most profitable and easy going day-to-day life for drivers, restaurants and delivery partners, through technology. To make all of this happen, we have a team of super talented professionals who interact in an environment full of creativity, energy and respect for diversity. The Orangers (our way of calling the employees) are the ones responsible for revolutionizing urban mobility (99), build quick opti

Secondly, let's create a list with all char used in main_responsabilities (PT), and job_other_details (PT) to understand which char we would like to remove.

In [55]:
#let's concatenate the text from all the rows together inserting a pipe | between lines
big_text =  str(dframe_jobs_final['job_main_responsabilities_pt'].str.cat(sep='|')) + \
            str(dframe_jobs_final['job_other_details_pt'].str.cat(sep='|'))
len(big_text)

134594

In [56]:
#Now let's create two different lists: one is for char + numbers from words only and the other for char from non_words only [punctuations, etc...]
char_from_words = []
char_from_non_words = []
for char in big_text:
    if ((char.isalpha()==True) or (char.isnumeric()==True)):
        char_from_words.append(char)
    else:
        char_from_non_words.append(char)

In [57]:
print("The total of char from words is: {words}".format(words=str(len(char_from_words))))
print("The total of char from non_words is: {non_words}".format(non_words=str(len(char_from_non_words))))

The total of char from words is: 112048
The total of char from non_words is: 22546


In [58]:
#Now let's remove duplicate char inside our lists of words and non_words and the sort them a-z
char_from_words_unique = list(dict.fromkeys(char_from_words))
char_from_non_words_unique = list(dict.fromkeys(char_from_non_words))
char_from_words_unique.sort()
char_from_non_words_unique.sort()

In [59]:
print("The total of unique char from words is: {words}".format(words=str(len(char_from_words_unique))))
print("The total of unique char from non_words is: {non_words}".format(non_words=str(len(char_from_non_words_unique))))

The total of unique char from words is: 77
The total of unique char from non_words is: 21


As we can see now, many of the chars aren't unique AT ALL!

In [60]:
#Let's check unique char from words
print(char_from_words_unique)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'º', 'Á', 'É', 'Ó', 'à', 'á', 'â', 'ã', 'ç', 'é', 'ê', 'í', 'ó', 'ô', 'õ', 'ú']


In [61]:
#Let's check unique char from non_words
print(char_from_non_words_unique)

['\n', ' ', '!', '"', '#', '&', '(', ')', '+', ',', '-', '.', '/', ':', ';', '|', '~', '·', '’', '•', '●']


In [62]:
#Removing & from list as it is necessary to form P&L which is 'DRE' in portuguese
char_from_non_words_unique.remove('&')

In [63]:
#Checking that & is out of the list
print(char_from_non_words_unique)

['\n', ' ', '!', '"', '#', '(', ')', '+', ',', '-', '.', '/', ':', ';', '|', '~', '·', '’', '•', '●']


Now let's import glossary_tech data in order to understand which positions require specific IT knowledges

In [247]:
dframe_glossary_tech = pd.read_excel('glossary_tech.xlsx',sheet_name='data')

In [248]:
dframe_glossary_tech.head()

,category_en,skill_en,description_en
0,Back-end Technologies Python,asyncio,A library to write concurrent code using the a...
1,Back-end Technologies Python,BeeWare,Collection of projects that can be used to hel...
2,Back-end Technologies Python,Bokeh,An interactive visualization library for Pytho...
3,Back-end Technologies Python,Bottle,"Fast, simple and lightweight WSGI micro web-fr..."
4,Back-end Technologies Python,Buildbot,Python-based software development continuous i...


In [249]:
# Remove punctuation
dframe_glossary_tech['skill_en_processed'] = dframe_glossary_tech['skill_en'].map(lambda x: re.sub('[-,\.!?]', '', x))

# Convert all to lowercase
dframe_glossary_tech['skill_en_processed'] = dframe_glossary_tech['skill_en_processed'].map(lambda x: x.lower())

# Removing all spaces in a separate column for tests
dframe_glossary_tech['skill_en_processed_no_spaces'] = dframe_glossary_tech['skill_en_processed'].map(lambda x: x.replace(' ',''))


In [250]:
skill_en_string = '|'.join(dframe_glossary_tech['skill_en'])

In [251]:
skill_pt_string = translate_text(target='pt-br',text=skill_en_string)

In [252]:
skill_pt_list = skill_pt_string.split('|')

In [253]:
dframe_glossary_tech['skill_pt'] = skill_pt_list

In [254]:
# Remove punctuation
dframe_glossary_tech['skill_pt_processed'] = dframe_glossary_tech['skill_pt'].map(lambda x: re.sub('[-,\.!?]', '', x))

# Convert all to lowercase
dframe_glossary_tech['skill_pt_processed'] = dframe_glossary_tech['skill_pt_processed'].map(lambda x: x.lower())

# Removing all spaces in a separate column for tests
dframe_glossary_tech['skill_pt_processed_no_spaces'] = dframe_glossary_tech['skill_pt_processed'].map(lambda x: x.replace(' ',''))

In [256]:
#exporting dframe_glossary_tech in order to analyze if all transaltions were performed
dframe_glossary_tech.to_csv('glossary_tech_processed.csv',index=False,sep='|', encoding='UTF-16')

In [257]:
#importing dframe glossary_tech after translations manually review, eg: Analysis wasn't translated to Análise, on the other hand, snowflake was translated to 'Floco de Neve'
dframe_glossary_tech_final = pd.read_excel('glossary_tech_manually_reviewed.xlsx')

In [258]:
dframe_glossary_tech_final.head()

,category_en,skill_en,description_en,skill_en_processed,skill_en_processed_no_spaces,skill_pt,skill_pt_processed,skill_pt_processed_no_spaces,comments
0,Back-end Technologies Python,asyncio,A library to write concurrent code using the a...,asyncio,asyncio,asyncio,asyncio,asyncio,NaN
1,Back-end Technologies Python,BeeWare,Collection of projects that can be used to hel...,beeware,beeware,BeeWare,beeware,beeware,NaN
2,Back-end Technologies Python,Bokeh,An interactive visualization library for Pytho...,bokeh,bokeh,Bokeh,bokeh,bokeh,NaN
3,Back-end Technologies Python,Bottle,"Fast, simple and lightweight WSGI micro web-fr...",bottle,bottle,Bottle,bottle,bottle,NaN
4,Back-end Technologies Python,Buildbot,Python-based software development continuous i...,buildbot,buildbot,Buildbot,buildbot,buildbot,NaN


Thirdly let's start processing our translated fields

In [72]:
# Remove punctuation
dframe_jobs_final['job_main_responsabilities_pt_processed'] = dframe_jobs_final['job_main_responsabilities_pt'].map(lambda x: re.sub('[-,\.!?]', '', x))
dframe_jobs_final['job_other_details_pt_processed'] = dframe_jobs_final['job_other_details_pt'].map(lambda x: re.sub('[-,\.!?]', '', x))

# Convert all to lowercase
dframe_jobs_final['job_main_responsabilities_pt_processed'] = dframe_jobs_final['job_main_responsabilities_pt_processed'].map(lambda x: x.lower())
dframe_jobs_final['job_other_details_pt_processed'] = dframe_jobs_final['job_other_details_pt_processed'].map(lambda x: x.lower())

#Removing consecutive spaces from both ends
dframe_jobs_final['job_main_responsabilities_pt_processed'] = dframe_jobs_final['job_main_responsabilities_pt_processed'].map(lambda x: x.strip())
dframe_jobs_final['job_other_details_pt_processed'] = dframe_jobs_final['job_other_details_pt_processed'].map(lambda x: x.strip())

In [73]:
dframe_jobs_final.iloc[0]

company_name                                                                             99
job_area                                                                            Finance
job_link                                  https://carreiras.99app.com/jobdetail/?vagaid=...
job_extraction_timestamp                                         2021-06-27 11:38:13.363000
job_about_company                         99 is a Brazilian technology company founded i...
job_title                                                      Finance and Planning Manager
job_location                                                                         Brazil
job_description                           We are looking for a seasoned FP&A Manager tha...
job_main_responsabilities                 -Detailed analysis of P&L variances versus for...
job_other_details                         - Strong expertise managing budget, forecast, ...
is_PCD_mentioned                                                                

Now let's start looking for 'job_area' in order to understand wheter we can combine areas or not

In [270]:
dframe_jobs_final.groupby(['job_area'],as_index=False)['job_extraction_timestamp'].count().sort_values(by=['job_area'],ascending=True)\
    #.to_csv('job_area.csv')

,job_area,job_extraction_timestamp
0,Adm & Finance,5
1,Audit,1
2,Auditoria e Compliance,2
3,Compliance,3
4,Contabilidade,6
5,Contabilidade/Controladoria,1
6,Contas a Receber,1
7,Controladoria,4
8,Credit & Formalization,1
9,Crédito,1


In [276]:
dframe_jobs_final.groupby(['job_title'],as_index=False)['job_extraction_timestamp'].count().sort_values(by=['job_title'],ascending=True)\
    #.to_csv('job_title.csv', encoding='UTF-16')

,job_title,job_extraction_timestamp
0,AML Specialist,1
1,Accounting Analyst Mexico,1
2,Analista Contábil Junior,1
3,Analista Contábil Pleno,1
4,Analista Contábil Sênior,2
...,...,...
91,Senior Liquidity and Funding Analyst,1
92,Senior M&A Analyst,1
93,Senior Strategic Planning Analyst (focused on ...,1
94,Tax Manager,1


In [278]:
dframe_jobs_final.groupby(['job_location'],as_index=False)['job_extraction_timestamp'].count().sort_values(by=['job_extraction_timestamp'],ascending=True)\
    #.to_csv('job_title.csv', encoding='UTF-16')

,job_location,job_extraction_timestamp
2,Chile,1
6,Portugal,1
7,Russia,1
8,United States,1
4,Japan,2
3,Colombia,3
1,Brazil or Remote,4
5,Mexico,11
0,Brazil,76


In [286]:
#Importing dict dframes
dframe_dict_jobarea = pd.read_excel('dict_job_area.xlsx')
dframe_dict_jobtitle = pd.read_excel('dict_job_title.xlsx')
dframe_dict_jobtitle = dframe_dict_jobtitle.drop(columns=['job_extraction_timestamp'])

In [287]:
dframe_dict_jobtitle.head()

,job_title,job_title_type
0,AML Specialist,specialist
1,Accounting Analyst Mexico,operational_JR
2,Analista Contábil Junior,operational_JR
3,Analista Contábil Pleno,operational_PL
4,Analista Contábil Sênior,operational_SR


In [288]:
dframe_dict_jobarea.head()

,Macro_job_area,Micro_job_area
0,Finance & Legal,Adm & Finance
1,Finance & Legal,Finance
2,Finance & Legal,Finance & Legal
3,Finance & Legal,Finance Operations - EBANX Pay
4,Finance & Legal,Finance and Legal


In [293]:
#Adding job_title_type
dframe_jobs_final = dframe_jobs_final.merge(dframe_dict_jobtitle, left_on='job_title', right_on='job_title')

In [296]:
#Adding macro_job_area
dframe_jobs_final = dframe_jobs_final.merge(dframe_dict_jobarea, left_on='job_area', right_on='Micro_job_area')

In [297]:
dframe_jobs_final.columns

Index(['company_name', 'job_area', 'job_link', 'job_extraction_timestamp',
       'job_about_company', 'job_title', 'job_location', 'job_description',
       'job_main_responsabilities', 'job_other_details', 'is_PCD_mentioned',
       'PCD_text', 'is_diversity_mentioned', 'Diversity_text', 'Comments',
       'job_description_lang', 'job_main_responsabilities_lang',
       'job_other_details_lang', 'job_main_responsabilities_pt',
       'job_other_details_pt', 'job_main_responsabilities_pt_processed',
       'job_other_details_pt_processed', 'job_title_type', 'Macro_job_area',
       'Micro_job_area'],
      dtype='object')

In [298]:
dframe_jobs_processed = dframe_jobs_final[['company_name',
                                            'job_area',
                                            'job_link',
                                            'job_extraction_timestamp',
                                            'job_title',
                                            'job_location',
                                            'job_main_responsabilities',
                                            'job_other_details',
                                            'job_main_responsabilities_lang',
                                            'job_other_details_lang',
                                            'job_main_responsabilities_pt',
                                            'job_other_details_pt',
                                            'job_main_responsabilities_pt_processed',
                                            'job_other_details_pt_processed',
                                            'job_title_type',
                                            'Macro_job_area']]

In [299]:
dframe_jobs_processed.head()

,company_name,job_area,job_link,job_extraction_timestamp,job_title,job_location,job_main_responsabilities,job_other_details,job_main_responsabilities_lang,job_other_details_lang,job_main_responsabilities_pt,job_other_details_pt,job_main_responsabilities_pt_processed,job_other_details_pt_processed,job_title_type,Macro_job_area
0,99,Finance,https://carreiras.99app.com/jobdetail/?vagaid=...,2021-06-27 11:38:13.363,Finance and Planning Manager,Brazil,-Detailed analysis of P&L variances versus for...,"- Strong expertise managing budget, forecast, ...",en,en,-Análise detalhada das variações de P&L versus...,- Forte experiência em gerenciamento de orçame...,análise detalhada das variações de p&l versus ...,forte experiência em gerenciamento de orçament...,management_manager,Finance & Legal
1,99,Finance,https://carreiras.99app.com/jobdetail/?vagaid=...,2021-06-27 11:42:08.465,Finance Business Partner,Colombia,Evaluate the end to end financial process for ...,"University graduate in the domain of Finance, ...",en,en,Avalie o processo financeiro de ponta a ponta ...,Graduação universitária no domínio de Finanças...,avalie o processo financeiro de ponta a ponta ...,graduação universitária no domínio de finanças...,business_partner,Finance & Legal
2,99,Finance,https://carreiras.99app.com/jobdetail/?vagaid=...,2021-06-27 11:44:29.556,Finance Manager,Mexico,"1. Financial Budget, Forecast, Closing, ...",The successful applicant:\nUniversity graduate...,en,en,"1. Orçamento Financeiro, Previsão, Fechamento,...",O candidato bem-sucedido: Graduado universitár...,1 orçamento financeiro previsão fechamento con...,o candidato bemsucedido: graduado universitári...,management_manager,Finance & Legal
3,99,Finance,https://carreiras.99app.com/jobdetail/?vagaid=...,2021-06-27 11:46:47.127,Finance Manager FP&A,Mexico,"1. Financial Budget, Forecast, Closing, ...","University graduate in Finance, Commerce or En...",en,en,"1. Orçamento Financeiro, Forecast, Fechamento,...","Graduação universitária em Finanças, Comércio ...",1 orçamento financeiro forecast fechamento con...,graduação universitária em finanças comércio o...,management_manager,Finance & Legal
4,99,Finance,https://carreiras.99app.com/jobdetail/?vagaid=...,2021-06-27 11:49:03.129,Finance Senior Manager,Mexico,"1. Financial Budget, Forecast, Closing, ...","University graduate in the domain of Finance, ...",en,en,"1. Orçamento Financeiro, Previsão, Fechamento,...","Graduado universitário no domínio de Finanças,...",1 orçamento financeiro previsão fechamento con...,graduado universitário no domínio de finanças ...,management_manager,Finance & Legal


In [300]:
#exporting processed data
dframe_jobs_processed.to_csv('dframe_jobs_processed.csv',index=False,sep='|',encoding='UTF-16')